In [9]:
'''
Feature Engineering
1. Convert Date to DaysOld
2. Dict Vectorizer of Categorical Variables (columns DaysOld, city, citygroup, type)
3. All the P columns stay AS-IS as numerical variables
Regression

1. Use Random Forest
'''
import pandas as pd
import numpy as np
from sklearn import ensemble, feature_extraction, preprocessing
from datetime import datetime

In [10]:
# import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sampleSubmission.csv')

In [20]:
# Convert OpenDate to DaysOld
def diff_dates_2015(date_x):
  date_format = "%m/%d/%Y"
  x = datetime.strptime(date_x, date_format)
  y = datetime.strptime('01/01/2015', date_format)
  delta = y - x
  return delta.days

train["days_old"] = train["Open Date"].apply(lambda x: diff_dates_2015(x))
test["days_old"] = test["Open Date"].apply(lambda x: diff_dates_2015(x))

In [21]:
# encode categorical variables
vec = feature_extraction.DictVectorizer()
names_categorical = []
for name in train.columns[2:-2] :    
    if name.startswith('P') == False:
        names_categorical.append(name)
        print name, len(np.unique(train[name]))
print names_categorical
X_train_cat = vec.fit_transform(train[names_categorical].T.to_dict().values()).todense()
X_test_cat = vec.transform(test[names_categorical].T.to_dict().values()).todense()
print X_train_cat.shape, X_test_cat.shape

City 34
City Group 2
Type 3
['City', 'City Group', 'Type']
(137, 39) (100000, 39)


In [22]:
# Numerical Labels
numerical_label = ['P'+str(i) for i in range(1,38)]
numerical_label.append('days_old')
X_train_num = train[numerical_label]
X_test_num = test[numerical_label]
print X_train_num.shape, X_test_num.shape

(137, 38) (100000, 38)


In [23]:
# Horizontally Stack - Categorical & Numerical Variables
X = np.hstack((X_train_cat, X_train_num))
Xt = np.hstack((X_test_cat,X_test_num))
y = train['revenue']
print X.shape, Xt.shape, y.shape

(137, 77) (100000, 77) (137,)


In [24]:
# Fit & Predict
regressor = ensemble.RandomForestRegressor(n_estimators=1000)
regressor.fit(X, y)
yp = regressor.predict(Xt)
print yp

[ 3795574.606  3428395.256  3250892.671 ...,  3545497.206  3503086.169
  5833691.437]


In [25]:
# create submission file
sample['Prediction']=yp
sample.to_csv('sample6.csv',index=False)